# Assistants API - Knowledge Retrieval 

In [2]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_: bool = load_dotenv(find_dotenv())  # read local .env file

In [3]:
client: OpenAI = OpenAI()

### Step 1: Upload the file and Create an Assistant

In [4]:
from openai.types.beta import Assistant

# Upload a file with an "assistants" purpose
file = client.files.create(file=open("zia_profile.pdf", "rb"), purpose="assistants")

print(file)

FileObject(id='file-kCbiOyEy7YInespbVKjd3CGC', bytes=48802, created_at=1700222301, filename='zia_profile.pdf', object='file', purpose='assistants', status='processed', status_details=None)


In [5]:
assistant: Assistant = client.beta.assistants.create(
    name="Student Support Assistant",
    instructions="You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia U. Khan.",
    model="gpt-3.5-turbo-1106",
    tools=[{"type": "retrieval"}],
    file_ids=[file.id],
)

### Step 2: Create a Thread

In [6]:
from openai.types.beta.thread import Thread

thread: Thread = client.beta.threads.create()

print(thread)

Thread(id='thread_PWEJVnptC835VBdF5njsY6Qj', created_at=1700222303, metadata={}, object='thread')


### Step 3: Add a Message to a Thread

In [7]:
from openai.types.beta.threads.thread_message import ThreadMessage

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="When and which city Zia U. Khan was born?",
)

### Step 4: Run the Assistant

In [8]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Pakistani. The user is the student of PIAIC.",
)

### Step 5: Check the Run status

In [11]:
run: Run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

print(run)

Run(id='run_mVymK7OEfpxoFYHV6b96GxQS', assistant_id='asst_AWrPlj801RKMzIf2OnpFzgFJ', cancelled_at=None, completed_at=1700222308, created_at=1700222304, expires_at=None, failed_at=None, file_ids=['file-kCbiOyEy7YInespbVKjd3CGC'], instructions='Please address the user as Pakistani. The user is the student of PIAIC.', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1700222304, status='completed', thread_id='thread_PWEJVnptC835VBdF5njsY6Qj', tools=[ToolAssistantToolsRetrieval(type='retrieval')])


### Step 6: Display the Assistant's Response

In [12]:
# from openai.resources.beta.threads.messages.messages import SyncCursorPage

messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)

for m in reversed(messages.data):
    print(m.role + ": " + m.content[0].text.value)

user: When and which city Zia U. Khan was born?
assistant: Zia U. Khan was born in Sialkot in 1961【7†source】.
